## Medium Orbit to GEO

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sgp4 import exporter, omm
from sgp4.api import Satrec
from astropy import units as u
from astropy.time import Time
from sgp4.api import days2mdhms
from sgp4.conveniences import sat_epoch_datetime
from sgp4.api import jday
from datetime import datetime, date, tzinfo, timezone, timedelta
from sgp4.api import SatrecArray

In [2]:
# read in the data

df=pd.read_json('./data/gp_space_track_03MAR2022.json',orient='records')

In [3]:
df[~(df['OBJECT_TYPE']=='PAYLOAD') & (df['EPOCH']>'2022-02-20T00:00:00') & (df['SEMIMAJOR_AXIS']>=8415)].shape

(2573, 40)

In [4]:
# creating the dataframe of non payload type members

dfnp=df[~(df['OBJECT_TYPE']=='PAYLOAD') & (df['EPOCH']>'2022-02-20T00:00:00') & (df['SEMIMAJOR_AXIS']>=8415) & (df['SEMIMAJOR_AXIS']<=50000)].copy(deep=True)

In [5]:
# creating payload members

dfbl=df[(df['OBJECT_TYPE']=='PAYLOAD') & (df['EPOCH']>'2022-02-20T00:00:00') & (df['SEMIMAJOR_AXIS']>=8415) & (df['SEMIMAJOR_AXIS']<=50000)].copy(deep=True)

In [6]:
dfbl.shape

(1753, 40)

In [7]:
dfnp.shape

(2564, 40)

In [8]:
dfbl.reset_index(drop=True,inplace=True)
ini_shape=dfbl.shape[0]
dfnp.reset_index(drop=True,inplace=True)
inis_shape=dfnp.shape[0]
dfnp.insert(0,'NEW_INDEX',list(range(ini_shape,ini_shape+inis_shape,1)))
dfnp.set_index('NEW_INDEX',inplace=True)
dfme=pd.concat([dfbl,dfnp])

In [9]:
%%time
# function creates the satellites from the TLE data, puts them in a list
# to be used later when using the array functionality in SGP4 library

def sat_list_func(dataframe):
    sats=[]
    for jj in range(dataframe.shape[0]):
        satellite = Satrec.twoline2rv(dataframe.iloc[jj].TLE_LINE1, dataframe.iloc[jj].TLE_LINE2)
        sats.append(satellite)
    return sats

Wall time: 0 ns


In [10]:
# distance function

def dist_func(psc_0,psc_1):
    dis_two=np.sqrt(sum((psc_1-psc_0)**2))
    return dis_two

In [11]:
# creation of the satellite array

sat_list_0=sat_list_func(dfme)

In [12]:
# using SatrecArray to create satellite array

sat_array_0 = SatrecArray(sat_list_0)

In [13]:
# need to create the time list
# at the moment we are using the 03MAR2022 list, lets start on the 4th instead

dt0 = datetime(2022,3,4,12,0,0)

In [14]:
nsd=60*60*24

In [15]:
# we will iterate in seconds but since SGP4 takes Julian dates
# need to convert to that format, this setup using datetime
# with the timedelta function will just let the user ask for
# x seconds in the future

# Also this code is currently setup under the assumption that
# time's are done in UTC, best to keep them in UTC and convert
# later is need be for other applications

In [16]:
# for 10 minutes of run time

In [17]:
%%time
time_steps_s=3*nsd
jda=[]
fra=[]
dt_time=[]
for _ in range(int(time_steps_s)):
    jd,fr=jday(dt0.year,dt0.month,dt0.day,dt0.hour,dt0.minute,dt0.second)
    jda.append(jd)
    fra.append(fr)
    dt_time.append(dt0)
    dt0=dt0+timedelta(seconds=1)

Wall time: 372 ms


In [18]:
%%time
# The satellites in this version will accept one time step per functional
# run

e, r, v = sat_array_0.sgp4(np.array([jda[0]]),np.array([fra[0]]))

Wall time: 2 ms


In [40]:
e.max()==0

True

In [19]:
# store the values in x
# still looking at sector searching as an option

x_vals=r[:,0,0]

In [20]:
# creation of the conjunction satellite dataframe

df_c=pd.DataFrame(columns=['SAT_0','SAT_1','DIST_KM','JDD','JDF','CONJ_TIME'])

In [21]:
# first second of the simulation, this will be collapsed into the main loop in later versions
# need to add back in the error checking line

In [23]:
%%time
t_val=[jda[0],fra[0],dt_time[0]]
# convert the r position to be a 2 dimensional instead of 3 dimensional
rf=r[:,0]
index=np.array(list(range(len(x_vals))))
# creating 4th column to append the index of the sat
rf=np.insert(rf,3,list(range(len(rf))),axis=1)
for j in range(0,dfbl.shape[0]):
    # check if satellites are next to one another
    delta=np.where((abs(x_vals[j+1:]-x_vals[j])<=10),True,False)
    index_slice=index[j+1:]
    #print(place_holder)
    #value index that is true ; b/c np.array->use masking + .index
    sat_conjunction_list=index_slice[delta].tolist()
    if sat_conjunction_list != []:
        for sat_c in sat_conjunction_list:
            dist=np.sqrt(sum((rf[sat_c][0:3]-rf[j][0:3])**2))
            if dist <=5:
                df_c.loc[len(df_c.index)] = [rf[j][3],rf[sat_c][3],dist,t_val[0],t_val[1],t_val[2]]

Wall time: 28 ms


In [24]:
df_c

,SAT_0,SAT_1,DIST_KM,JDD,JDF,CONJ_TIME
0,965.0,1681.0,0.18723,2459642.5,0.5,2022-03-04 12:00:00


In [25]:
# main loop
# need to add back in the error checking line

In [26]:
%%time
count = 1
for ITER in range(1,len(jda)):
    count+=1
    e, r, v = sat_array_0.sgp4(np.array([jda[ITER]]),np.array([fra[ITER]]))

    x_vals=r[:,0,0]

    t_val=[jda[ITER],fra[ITER],dt_time[ITER]]
    # convert the r position to be a 2 dimensional instead of 3 dimensional
    rf=r[:,0]
    # creating 4th column to append the index of the sat
    rf=np.insert(rf,3,list(range(len(rf))),axis=1)
    for j in range(0,dfbl.shape[0]):
        # check if satellites are next to one another
        delta=np.where((abs(x_vals[j+1:]-x_vals[j])<=10),True,False)
        index_slice=index[j+1:]
        # value index that is true ; b/c np.array->use masking + .index
        sat_conjunction_list=index_slice[delta].tolist()
        if sat_conjunction_list != []:
            for sat_c in sat_conjunction_list:
                dist=np.sqrt(sum((rf[sat_c][0:3]-rf[j][0:3])**2))
                if dist <=5:
                    df_c.loc[len(df_c.index)] = [rf[j][3],rf[sat_c][3],dist,t_val[0],t_val[1],t_val[2]]
    if np.mod(count,100)==0: 
        df_c.sort_values('DIST_KM',inplace=True)
        df_c.drop_duplicates(subset=['SAT_0','SAT_1'], keep='first', inplace=True, ignore_index=True)

Wall time: 2h 53s


In [27]:
df_c.sort_values('DIST_KM',inplace=True)
df_c.drop_duplicates(subset=['SAT_0','SAT_1'], keep='first', inplace=True, ignore_index=True)
df_c.head(5)

,SAT_0,SAT_1,DIST_KM,JDD,JDF,CONJ_TIME
0,965.0,1681.0,0.042911,2459642.5,0.939259,2022-03-04 22:32:32
1,412.0,2110.0,0.743144,2459644.5,0.372338,2022-03-06 08:56:10
2,1561.0,3632.0,1.019166,2459644.5,0.329722,2022-03-06 07:54:48
3,1600.0,1656.0,2.906344,2459642.5,0.531493,2022-03-04 12:45:21
4,1433.0,1515.0,3.474887,2459645.5,0.233507,2022-03-07 05:36:15


In [28]:
df_c['SAT_0_NAME']=df_c['SAT_0'].apply(lambda x: dfme.iloc[int(x)].OBJECT_NAME)

In [29]:
df_c['SAT_1_NAME']=df_c['SAT_1'].apply(lambda x: dfme.iloc[int(x)].OBJECT_NAME)

In [30]:
df_c.head(5)

,SAT_0,SAT_1,DIST_KM,JDD,JDF,CONJ_TIME,SAT_0_NAME,SAT_1_NAME
0,965.0,1681.0,0.042911,2459642.5,0.939259,2022-03-04 22:32:32,INTELSAT 901,MEV-1
1,412.0,2110.0,0.743144,2459644.5,0.372338,2022-03-06 08:56:10,COSMOS 1779 (GLONASS),SL-6 R/B(2)
2,1561.0,3632.0,1.019166,2459644.5,0.329722,2022-03-06 07:54:48,GSAT 19,COSMOS 1700 DEB
3,1600.0,1656.0,2.906344,2459642.5,0.531493,2022-03-04 12:45:21,O3B FM13,O3B FM19
4,1433.0,1515.0,3.474887,2459645.5,0.233507,2022-03-07 05:36:15,INTELSAT 30,INTELSAT 31


In [31]:
df_c.columns

Index(['SAT_0', 'SAT_1', 'DIST_KM', 'JDD', 'JDF', 'CONJ_TIME', 'SAT_0_NAME',
       'SAT_1_NAME'],
      dtype='object')

In [ ]:
e.max()==0

In [32]:
# final test data frame!

In [33]:
df_c[['SAT_0_NAME','SAT_1_NAME','DIST_KM','CONJ_TIME','SAT_0','SAT_1','JDD','JDF']].head(15)

,SAT_0_NAME,SAT_1_NAME,DIST_KM,CONJ_TIME,SAT_0,SAT_1,JDD,JDF
0,INTELSAT 901,MEV-1,0.042911,2022-03-04 22:32:32,965.0,1681.0,2459642.5,0.939259
1,COSMOS 1779 (GLONASS),SL-6 R/B(2),0.743144,2022-03-06 08:56:10,412.0,2110.0,2459644.5,0.372338
2,GSAT 19,COSMOS 1700 DEB,1.019166,2022-03-06 07:54:48,1561.0,3632.0,2459644.5,0.329722
3,O3B FM13,O3B FM19,2.906344,2022-03-04 12:45:21,1600.0,1656.0,2459642.5,0.531493
4,INTELSAT 30,INTELSAT 31,3.474887,2022-03-07 05:36:15,1433.0,1515.0,2459645.5,0.233507
5,OPS 1570,INMARSAT 3-F4,4.279183,2022-03-04 12:03:31,93.0,800.0,2459642.5,0.502442
6,COMS 1,GEO-KOMPSAT-2A,4.502777,2022-03-06 10:50:13,1253.0,1641.0,2459644.5,0.451539
7,LDPE-1,ASCENT,4.600260,2022-03-04 15:17:18,1745.0,1752.0,2459642.5,0.637014
8,COSMOS 2442 (GLONASS),COSMOS 2478 (GLONASS),4.823506,2022-03-04 16:00:07,1191.0,1316.0,2459642.5,0.666748


In [34]:
df_c.tail(5)

,SAT_0,SAT_1,DIST_KM,JDD,JDF,CONJ_TIME,SAT_0_NAME,SAT_1_NAME
4,1433.0,1515.0,3.474887,2459645.5,0.233507,2022-03-07 05:36:15,INTELSAT 30,INTELSAT 31
5,93.0,800.0,4.279183,2459642.5,0.502442,2022-03-04 12:03:31,OPS 1570,INMARSAT 3-F4
6,1253.0,1641.0,4.502777,2459644.5,0.451539,2022-03-06 10:50:13,COMS 1,GEO-KOMPSAT-2A
7,1745.0,1752.0,4.600260,2459642.5,0.637014,2022-03-04 15:17:18,LDPE-1,ASCENT
8,1191.0,1316.0,4.823506,2459642.5,0.666748,2022-03-04 16:00:07,COSMOS 2442 (GLONASS),COSMOS 2478 (GLONASS)


In [35]:
df_c.to_csv('./data/medium_orbit_3days_MAR3GPDATA.csv',index=False)

In [36]:
# ------------------------------------------------------------------------------------------------------

In [37]:
r[:,0,0]

array([-7259.19777555, -6778.83233983, -2251.55626972, ...,
        3254.02239369,  5832.60352645,  6213.43718042])